In [1]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

2024-05-22 19:18:57.998741: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 19:18:57.998793: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 19:18:58.000271: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# load data frames
df_train = pd.read_csv('/kaggle/input/dataframes-bal/train_bal.csv')
df_val = pd.read_csv('/kaggle/input/dataframes-bal/val_bal.csv')
df_test = pd.read_csv('/kaggle/input/dataframes-bal/test_bal.csv')

# load joint embeddings
#with open('/kaggle/input/j-embeds-bal/j_embeds_train (1).pkl', 'rb') as f:
    #v_embeds_train = pickle.load(f) 
    
#with open('/kaggle/input/v-embeds/v_embeds_val.pkl', 'rb') as f:
  #  v_embeds_val = pickle.load(f) 
    
with open('/kaggle/input/j-embeds-bal/j_embeds_test (2).pkl', 'rb') as f:
    j_embeds_test = pickle.load(f) 

In [3]:
print(len(j_embeds_test)) # Liste aus Tensors
print(j_embeds_test[0].shape)
print(type(j_embeds_test[0]))
print(j_embeds_test[0].device)

89
torch.Size([612, 768])
<class 'torch.Tensor'>
cuda:0


In [4]:
### TEST: Concatentate test tensor along rows (first dimension) --> if that works, process and save training j embeds as chunks
batches = int(len(df_train)/len(df_test)) # 18 batches
test_embeds = torch.stack(j_embeds_test, dim=0) # stack it first, optionally:push to CPU and use an operation for concatenating numpy arrays

concatenated_tensor = test_embeds # start with normal tensor
for i in range(batches):
    concatenated_tensor = torch.cat((concatenated_tensor, test_embeds), dim=0) # add another batch

print("Concatenated tensor along rows:")

print(concatenated_tensor.shape) # LÖSUNG: j_embeds_train in chunks extrahieren und speichern, dann unten (eine Zelle weiter) auf cpu pushen. Reicht aus!
# geht nut einmal, dann cuda out of memory
concatenated_tensor = concatenated_tensor.cpu().numpy()
print(concatenated_tensor.shape)

Concatenated tensor along rows:
torch.Size([1691, 612, 768])
(1691, 612, 768)


In [5]:
X_train = torch.stack(j_embeds_test[:70], dim=0).cpu().numpy()
X_val = torch.stack(j_embeds_test[70:80], dim=0).cpu().numpy()
X_test = torch.stack(j_embeds_test[80:89], dim=0).cpu().numpy()
print(X_train.shape, X_val.shape, X_test.shape)

(70, 612, 768) (10, 612, 768) (9, 612, 768)


In [6]:
# convert labels to numerical values
label_encoder = LabelEncoder()
labels = df_test['AUTHOR'].tolist()
y_train = labels[:70]
y_val = labels[70:80]
y_test = labels[80:89]

y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)
print(y_test)

[9 5 1 3 8 4 6 9 9]


In [7]:
# Define the model
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(612, 768)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val)) # decrease epochs if needed

# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')



Epoch 1/20
1/2 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0781 - loss: 2.4064

I0000 00:00:1716405567.191020    3006 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1716405567.203811    3006 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.0737 - loss: 4.8592 - val_accuracy: 0.2000 - val_loss: 44.6265
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.1369 - loss: 45.7979 - val_accuracy: 0.1000 - val_loss: 56.2127
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.1906 - loss: 51.3644 - val_accuracy: 0.2000 - val_loss: 55.2003
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.2929 - loss: 37.6575 - val_accuracy: 0.2000 - val_loss: 23.9204
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.5728 - loss: 13.2881 - val_accuracy: 0.2000 - val_loss: 20.2950
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.6759 - loss: 5.5925 - val_accuracy: 0.2000 - val_loss: 24.2889
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.7253 - loss: 2.8993 - val_accuracy: 0.2000 - val_loss: 28.4521
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.6759 - loss: 3.3770 - val_accuracy: 0.3000 - val_loss: 22.3780
E

Besser als mit visuellen Features und über Chance Performance. Allerdings nur mit 109 Datensätzen trainiert. Wie kann man sichergehen, dass das Model nicht nur die Textfeatures benutzt? 